### This modified notebook will downlaod the tiff files off 'https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-pre-c1-l2a/21/N/YC/2022/12/S2B_T21NYC_20221205T140704_L2A/TCI.tif', show the projection, reproject them in the next block. A progress bar is added as well. (check the screenshots!)

In [2]:
import os
import rasterio
import urllib.request
import pystac
import rasterio

from shapely.geometry import Polygon, mapping
from datetime import datetime, timezone
from pystac.extensions.eo import Band, EOExtension
from tempfile import TemporaryDirectory

In [5]:
import os
import rasterio
import urllib.request
import pystac

from shapely.geometry import Polygon, mapping
from datetime import datetime, timezone
from pystac.extensions.eo import Band, EOExtension
from tempfile import TemporaryDirectory

In [ ]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

# Set temporary directory to store source data
tmp_dir = TemporaryDirectory()
img_path1 = os.path.join(tmp_dir.name, 'image1.tif')

# Fetch and store data
url1 = ('https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-pre-c1-l2a/21/N/YC/2022/12/S2B_T21NYC_20221205T140704_L2A/TCI.tif')

print("Downloading")
with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc="Downloading Sentinel-2 Image") as t:
    urllib.request.urlretrieve(url1, img_path1, reporthook=t.update_to)

print("Download complete!")
print("img_path1:", img_path1, "\n")

# Extract and print projection
with rasterio.open(img_path1) as dataset:
    print("Image Projection:", dataset.crs)


Downloading


In [23]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Define output path for reprojected image
reprojected_img_path = os.path.join(tmp_dir.name, 'image1_reprojected.tif')

# Open the original dataset
with rasterio.open(img_path1) as src:
    # Define target CRS
    dst_crs = "EPSG:4326"
    
    # Calculate transform and new dimensions
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    # Define metadata for output file
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    # Reproject and save to new file
    with rasterio.open(reprojected_img_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )

print("Reprojection complete!")
print("Reprojected image saved at:", reprojected_img_path)

# Verify new projection
with rasterio.open(reprojected_img_path) as dst:
    print("New Image Projection:", dst.crs)


Reprojection complete!
Reprojected image saved at: C:\Users\shubh\AppData\Local\Temp\tmpftjiy7_s\image1_reprojected.tif
New Image Projection: EPSG:4326


In [7]:
def get_bbox_and_footprint(raster):
    with rasterio.open(raster) as r:
        bounds = r.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [8]:
# Run the function and print out the results for image 1
bbox, footprint = get_bbox_and_footprint(img_path1)
print("bbox: ", bbox, "\n")
print("footprint: ", footprint)

bbox:  [37.6616853489879, 55.73478197572927, 37.66573047610874, 55.73882710285011] 

footprint:  {'type': 'Polygon', 'coordinates': (((37.6616853489879, 55.73478197572927), (37.6616853489879, 55.73882710285011), (37.66573047610874, 55.73882710285011), (37.66573047610874, 55.73478197572927), (37.6616853489879, 55.73478197572927)),)}


In [9]:
# Run the function and print out the results for image 2
bbox2, footprint2 = get_bbox_and_footprint(img_path2)
print("bbox: ", bbox2, "\n")
print("footprint: ", footprint2)

bbox:  [37.67786535472783, 55.726691972859314, 37.68191048184866, 55.730737099980146] 

footprint:  {'type': 'Polygon', 'coordinates': (((37.67786535472783, 55.726691972859314), (37.67786535472783, 55.730737099980146), (37.68191048184866, 55.730737099980146), (37.68191048184866, 55.726691972859314), (37.67786535472783, 55.726691972859314)),)}


In [10]:
wv3_bands = [Band.create(name='Coastal', description='Coastal: 400 - 450 nm', common_name='coastal'),
             Band.create(name='Blue', description='Blue: 450 - 510 nm', common_name='blue'),
             Band.create(name='Green', description='Green: 510 - 580 nm', common_name='green'),
             Band.create(name='Yellow', description='Yellow: 585 - 625 nm', common_name='yellow'),
             Band.create(name='Red', description='Red: 630 - 690 nm', common_name='red'),
             Band.create(name='Red Edge', description='Red Edge: 705 - 745 nm', common_name='rededge'),
             Band.create(name='Near-IR1', description='Near-IR1: 770 - 895 nm', common_name='nir08'),
             Band.create(name='Near-IR2', description='Near-IR2: 860 - 1040 nm', common_name='nir09')]

In [11]:
wv3_bands = [Band.create(name='Coastal', description='Coastal: 400 - 450 nm', common_name='coastal'),
             Band.create(name='Blue', description='Blue: 450 - 510 nm', common_name='blue'),
             Band.create(name='Green', description='Green: 510 - 580 nm', common_name='green'),
             Band.create(name='Yellow', description='Yellow: 585 - 625 nm', common_name='yellow'),
             Band.create(name='Red', description='Red: 630 - 690 nm', common_name='red'),
             Band.create(name='Red Edge', description='Red Edge: 705 - 745 nm', common_name='rededge'),
             Band.create(name='Near-IR1', description='Near-IR1: 770 - 895 nm', common_name='nir08'),
             Band.create(name='Near-IR2', description='Near-IR2: 860 - 1040 nm', common_name='nir09')]

In [12]:
collection_item = pystac.Item(id='local-image-col-1',
                               geometry=footprint,
                               bbox=bbox,
                               datetime=datetime.utcnow(),
                               properties={})

collection_item.common_metadata.gsd = 0.3
collection_item.common_metadata.platform = 'Maxar'
collection_item.common_metadata.instruments = ['WorldView3']

asset = pystac.Asset(href=img_path1, 
                      media_type=pystac.MediaType.GEOTIFF)
collection_item.add_asset("image", asset)
eo = EOExtension.ext(collection_item.assets["image"], add_if_missing=True)
eo.apply(wv3_bands)

collection_item2 = pystac.Item(id='local-image-col-2',
                               geometry=footprint2,
                               bbox=bbox2,
                               datetime=datetime.utcnow(),
                               properties={})

collection_item2.common_metadata.gsd = 0.3
collection_item2.common_metadata.platform = 'Maxar'
collection_item2.common_metadata.instruments = ['WorldView3']

asset2 = pystac.Asset(href=img_path2,
                     media_type=pystac.MediaType.GEOTIFF)
collection_item2.add_asset("image", asset2)
eo = EOExtension.ext(collection_item2.assets["image"], add_if_missing=True)
eo.apply([
    band for band in wv3_bands if band.name in ["Red", "Green", "Blue"]
])

In [13]:
from shapely.geometry import shape

unioned_footprint = shape(footprint).union(shape(footprint2))
collection_bbox = list(unioned_footprint.bounds)
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

In [14]:
collection_interval = sorted([collection_item.datetime, collection_item2.datetime])
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])

In [15]:
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [16]:
collection = pystac.Collection(id='wv3-images',
                               description='Spacenet 5 images over Moscow',
                               extent=collection_extent,
                               license='CC-BY-SA-4.0')

In [17]:
collection.add_items([collection_item, collection_item2])
catalog = pystac.Catalog(id='catalog-with-collection', 
                         description='This Catalog is a basic demonstration of how to include a Collection in a STAC Catalog.')
catalog.add_child(collection)

<Link rel=child target=<Collection id=wv3-images>>

In [18]:
catalog.describe()

* <Catalog id=catalog-with-collection>
    * <Collection id=wv3-images>
      * <Item id=local-image-col-1>
      * <Item id=local-image-col-2>


In [19]:
catalog.normalize_and_save(root_href=os.path.join(tmp_dir.name, 'stac-collection'), 
                           catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [4]:
import os
import rasterio
from rasterio.warp import transform_bounds
import urllib.request
import pystac
from shapely.geometry import Polygon, mapping
from datetime import datetime, timezone
from pystac.extensions.eo import Band, EOExtension
from tempfile import TemporaryDirectory

def get_tiff_metadata(tiff_path):
    """
    Extract bbox and assets information from a GeoTIFF file.
    
    Args:
        tiff_path (str): Path to the GeoTIFF file
        
    Returns:
        dict: Dictionary containing bbox and assets information
    """
    with rasterio.open(tiff_path) as dataset:
        # Get the bounds in the original CRS
        bounds = dataset.bounds
        
        # Transform bounds from original CRS to WGS84 (EPSG:4326)
        bbox = transform_bounds(dataset.crs, 'EPSG:4326',
                              bounds.left, bounds.bottom,
                              bounds.right, bounds.top)
        
        # Create the metadata structure
        metadata = {
            "assets": {
                "image": {
                    "href": tiff_path,
                    "type": "image/tiff; application=geotiff"
                }
            },
            "bbox": list(bbox),
            "stac_extensions": []
        }
        
        return metadata

tiff_path = "https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-pre-c1-l2a/21/N/YC/2022/12/S2B_T21NYC_20221205T140704_L2A/TCI.tif"
try:
    metadata = get_tiff_metadata(tiff_path)
    print(metadata)
except Exception as e:
    print(f"Error processing file: {e}")

{'assets': {'image': {'href': 'https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-pre-c1-l2a/21/N/YC/2022/12/S2B_T21NYC_20221205T140704_L2A/TCI.tif', 'type': 'image/tiff; application=geotiff'}}, 'bbox': [-55.202501808647625, 1.7187506083549096, -54.214267703874334, 2.7128294056121574], 'stac_extensions': []}
